#### Well done! Having spent years analyzing financial records for big banks, you've finally scratched your idealistic itch and joined the education sector. In your latest role, you've become the Chief Data Scientist for your city's school district. In this capacity, you'll be helping the  school board and mayor make strategic decisions regarding future school budgets and priorities.

As a first task, you've been asked to analyze the district-wide standardized test results. You'll be given access to every student's math and reading scores, as well as various information on the schools they attend. Your responsibility is to aggregate the data to and showcase obvious trends in school 

In [1]:
#import dependencies
import pandas as pd
import numpy as np
import os

In [24]:
# load csv files located in the resources folder
school_data = os.path.join('Resources','schools_complete.csv')
student_data = os.path.join('Resources', 'students_complete.csv')


# Read each csv into a data frame (df)
school_data_df = pd.read_csv(school_data)
student_data_df = pd.read_csv(student_data)

# Combine into single df  
school_data_merged = student_data_df.merge(school_data_df, how="left", on="school_name")

In [25]:
school_data_merged.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


### District Summary

* Create a high level snapshot (in table form) of the district's key metrics, including:
  * Total Schools
  * Total Students
  * Total Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math (The percentage of students that passed math.)
  * % Passing Reading (The percentage of students that passed reading.)
  * % Overall Passing (The percentage of students that passed math **and** reading.)

In [30]:
#as a list, the total number of 'unique' school_name data points in the merged df
school_data_names = school_data_merged['school_name'].unique()

#then return a count of the number of unique data points in the list
school_count = len(school_data_names)

#calculate the number of total students
school_stu_total = school_data_merged['student_name'].count()

#total budget for all schools
school_bdgt = school_data_merged['budget'].sum()

#avg math score
avg_math = school_data_merged['math_score'].mean()

#avg reading score
avg_read = school_data_merged['reading_score'].mean()

# % passing math by creating a variable for the number of students that pass with at least a 70 using .loc
math_passing_score = len(school_data_merged.loc[school_data_merged['math_score'] >= 70])
pct_math_passing = math_passing_score/school_stu_total

# % passing reading by creating a variable for the number of students that pass with at least a 70 using .loc
read_passing_score = len(school_data_merged.loc[school_data_merged['reading_score'] >= 70])
pct_read_passing = read_passing_score/school_stu_total

#overall passing
tot_pass_pct = student_data_df[(student_data_df['math_score'] >= 70) & (student_data_df['reading_score'] >= 70)]['student_name'].count()/school_stu_total


In [31]:
#create the df for district data
district_df = pd.DataFrame({
    "Total # of Schools":[school_count],
    "Total # of Students":[school_stu_total],
    "Total School Budget":[school_bdgt],
    "Average Math Score":[avg_math],
    "Pct Passing Math":[pct_math_passing],
    "Average Reading Score":[avg_read],
    "Pct Passing Reading":[pct_read_passing],
    "Overall Passing Pct":[tot_pass_pct]
})

In [32]:
district_df

,Total # of Schools,Total # of Students,Total School Budget,Average Math Score,Pct Passing Math,Average Reading Score,Pct Passing Reading,Overall Passing Pct
0,15,39170,82932329558,78.985371,0.749809,81.87784,0.858055,0.651723


In [23]:
#render styles for cells and hide the index, which is of no data use
district_df.style.format({"Total School Budget": "${:20,.0f}",
                         "Average Math Score":"{:.2f}",
                          "Average Reading Score":"{:.2f}",
                          "Pct Passing Math":"{:.1%}",
                          "Pct Passing Reading":"{:.1%}",
                          "Overall Passing Pct":"{:.1%}"})\
.hide_index()

Total # of Schools,Total # of Students,Total School Budget,Average Math Score,Pct Passing Math,Average Reading Score,Pct Passing Reading,Overall Passing Pct
15,39170,"$ 82,932,329,558",78.99,75.0%,81.88,85.8%,65.2%


### School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math (The percentage of students that passed math.)
  * % Passing Reading (The percentage of students that passed reading.)
  * % Overall Passing (The percentage of students that passed math **and** reading.)

In [8]:
# indexed by school
indexed_school = school_data_merged.set_index('school_name').groupby(['school_name'])

# type of school
type_of_school = indexed_school['type'].first()

#total # of student per school
total_stu_per_school = indexed_school.size()

# individual school budget
ind_school_bdgt = indexed_school['budget'].first()

# per student budget
per_stu_bdgt = ind_school_bdgt/total_stu_per_school

# avg math score per school
avg_math_per = indexed_school['math_score'].mean()

# avg reading score per school
avg_read_per = indexed_school['reading_score'].mean()

# passing math rate/school
pass_math_per = school_data_merged[school_data_merged['math_score']>=70].groupby('school_name')['Student ID'].count()/total_stu_per_school

# passing reading rate/school
pass_read_per = school_data_merged[school_data_merged['reading_score']>=70].groupby('school_name')['Student ID'].count()/total_stu_per_school

# overall passing rate/school, easier math
pct_overall_per = (pass_math_per + pass_read_per)/2

### create the df for individual school data

ind_school_summary = pd.DataFrame({
    "Type of School": type_of_school,
    "Total # of Students": total_stu_per_school,
    "Total School Budget": ind_school_bdgt,
    "Per Student Budget": per_stu_bdgt,
    "Average Math Score": avg_math_per,
    "Passing Math Rate": pass_math_per,
    "Average Reading Score": avg_read_per,
    "Passing Reading Rate": pass_read_per,
    "Overall Passing Rate": pct_overall_per
})

#formatting
ind_school_summary.style.format({'Total # of Students': '{:,}', 
                          "Total School Budget": "${:20,.0f}", 
                          "Per Student Budget": "${:20,.0f}",
                          'Average Math Score': "{:.2f}", 
                          'Average Reading Score': "{:.2f}", 
                          "Passing Math Rate": "{:.1%}", 
                          "Passing Reading Rate": "{:.1%}", 
                          "Overall Passing Rate": "{:.1%}"})

,Type of School,Total # of Students,Total School Budget,Per Student Budget,Average Math Score,Passing Math Rate,Average Reading Score,Passing Reading Rate,Overall Passing Rate
school_name,,,,,,,,,
Bailey High School,District,"4,976","$ 3,124,928",$ 628,77.05,66.7%,81.03,81.9%,74.3%
Cabrera High School,Charter,"1,858","$ 1,081,356",$ 582,83.06,94.1%,83.98,97.0%,95.6%
Figueroa High School,District,"2,949","$ 1,884,411",$ 639,76.71,66.0%,81.16,80.7%,73.4%
Ford High School,District,"2,739","$ 1,763,916",$ 644,77.10,68.3%,80.75,79.3%,73.8%
Griffin High School,Charter,"1,468","$ 917,500",$ 625,83.35,93.4%,83.82,97.1%,95.3%
Hernandez High School,District,"4,635","$ 3,022,020",$ 652,77.29,66.8%,80.93,80.9%,73.8%
Holden High School,Charter,427,"$ 248,087",$ 581,83.80,92.5%,83.81,96.3%,94.4%
Huang High School,District,"2,917","$ 1,910,635",$ 655,76.63,65.7%,81.18,81.3%,73.5%
Johnson High School,District,"4,761","$ 3,094,650",$ 650,77.07,66.1%,80.97,81.2%,73.6%


### Top Performing Schools (By % Overall Passing)

* Create a table that highlights the top 5 performing schools based on % Overall Passing. Include:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math (The percentage of students that passed math.)
  * % Passing Reading (The percentage of students that passed reading.)
  * % Overall Passing (The percentage of students that passed math **and** reading.)

In [9]:
# by passing rate
top_schools = ind_school_summary.sort_values("Overall Passing Rate", ascending = False)
# return the top 5
top_schools.head().style.format({'Total # of Students': '{:,}', 
                          "Total School Budget": "${:,}", 
                          "Per Student Budget": "${:.0f}",
                          'Average Math Score': "{:.1f}", 
                          'Average Reading Score': "{:.1f}", 
                          "Passing Math Rate": "{:.1%}", 
                          "Passing Reading Rate": "{:.1%}", 
                          "Overall Passing Rate": "{:.1%}"})

,Type of School,Total # of Students,Total School Budget,Per Student Budget,Average Math Score,Passing Math Rate,Average Reading Score,Passing Reading Rate,Overall Passing Rate
school_name,,,,,,,,,
Cabrera High School,Charter,"1,858","$1,081,356",$582,83.1,94.1%,84.0,97.0%,95.6%
Thomas High School,Charter,"1,635","$1,043,130",$638,83.4,93.3%,83.8,97.3%,95.3%
Pena High School,Charter,962,"$585,858",$609,83.8,94.6%,84.0,95.9%,95.3%
Griffin High School,Charter,"1,468","$917,500",$625,83.4,93.4%,83.8,97.1%,95.3%
Wilson High School,Charter,"2,283","$1,319,574",$578,83.3,93.9%,84.0,96.5%,95.2%


### Bottom Performing Schools (By % Overall Passing)

* Create a table that highlights the bottom 5 performing schools based on % Overall Passing. Include all of the same metrics as above.

In [10]:
#use the tail() function to get the bottom 5
worst_performing = top_schools.tail()

#sort so the worst is now on top
worst_performing = worst_performing.sort_values("Overall Passing Rate")

#format
worst_performing.head().style.format({'Total # of Students': '{:,}', 
                          "Total School Budget": "${:,}", 
                          "Per Student Budget": "${:.0f}",
                          'Average Math Score': "{:.1f}", 
                          'Average Reading Score': "{:.1f}", 
                          "Passing Math Rate": "{:.1%}", 
                          "Passing Reading Rate": "{:.1%}", 
                          "Overall Passing Rate": "{:.1%}"})

,Type of School,Total # of Students,Total School Budget,Per Student Budget,Average Math Score,Passing Math Rate,Average Reading Score,Passing Reading Rate,Overall Passing Rate
school_name,,,,,,,,,
Rodriguez High School,District,"3,999","$2,547,363",$637,76.8,66.4%,80.7,80.2%,73.3%
Figueroa High School,District,"2,949","$1,884,411",$639,76.7,66.0%,81.2,80.7%,73.4%
Huang High School,District,"2,917","$1,910,635",$655,76.6,65.7%,81.2,81.3%,73.5%
Johnson High School,District,"4,761","$3,094,650",$650,77.1,66.1%,81.0,81.2%,73.6%
Ford High School,District,"2,739","$1,763,916",$644,77.1,68.3%,80.7,79.3%,73.8%


### Math Scores by Grade\*\*

* Create a table that lists the average Math Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

In [11]:
#avg per grade
math_nine = student_data_df.loc[student_data_df['grade'] == '9th'].groupby('school_name')["math_score"].mean()
math_ten = student_data_df.loc[student_data_df['grade'] == '10th'].groupby('school_name')["math_score"].mean()
math_eleven = student_data_df.loc[student_data_df['grade'] == '11th'].groupby('school_name')["math_score"].mean()
math_twelve = student_data_df.loc[student_data_df['grade'] == '12th'].groupby('school_name')["math_score"].mean()

#create the df
math_avg = pd.DataFrame({
        "9th": math_nine,
        "10th": math_ten,
        "11th": math_eleven,
        "12th": math_twelve
})

math_avg = math_avg[['9th', '10th', '11th', '12th']]
math_avg.index.name = "School"

#format
math_avg.style.format({'9th': '{:.1f}', 
                          "10th": '{:.1f}', 
                          "11th": "{:.1f}", 
                          "12th": "{:.1f}"})

,9th,10th,11th,12th
School,,,,
Bailey High School,77.1,77.0,77.5,76.5
Cabrera High School,83.1,83.2,82.8,83.3
Figueroa High School,76.4,76.5,76.9,77.2
Ford High School,77.4,77.7,76.9,76.2
Griffin High School,82.0,84.2,83.8,83.4
Hernandez High School,77.4,77.3,77.1,77.2
Holden High School,83.8,83.4,85.0,82.9
Huang High School,77.0,75.9,76.4,77.2
Johnson High School,77.2,76.7,77.5,76.9


### Reading Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.


In [12]:
#avg reading per grade
read_nine = student_data_df.loc[student_data_df['grade'] == '9th'].groupby('school_name')["reading_score"].mean()
read_ten = student_data_df.loc[student_data_df['grade'] == '10th'].groupby('school_name')["reading_score"].mean()
read_eleven = student_data_df.loc[student_data_df['grade'] == '11th'].groupby('school_name')["reading_score"].mean()
read_twelve = student_data_df.loc[student_data_df['grade'] == '12th'].groupby('school_name')["reading_score"].mean()

read_avg = pd.DataFrame({
        "9th": read_nine,
        "10th": read_ten,
        "11th": read_eleven,
        "12th": read_twelve
})
read_avg = read_avg[['9th', '10th', '11th', '12th']]
read_avg.index.name = "School"

#show and format
read_avg.style.format({'9th': '{:.1f}', 
                          "10th": '{:.1f}', 
                          "11th": "{:.1f}", 
                          "12th": "{:.1f}"})

,9th,10th,11th,12th
School,,,,
Bailey High School,81.3,80.9,80.9,80.9
Cabrera High School,83.7,84.3,83.8,84.3
Figueroa High School,81.2,81.4,80.6,81.4
Ford High School,80.6,81.3,80.4,80.7
Griffin High School,83.4,83.7,84.3,84.0
Hernandez High School,80.9,80.7,81.4,80.9
Holden High School,83.7,83.3,83.8,84.7
Huang High School,81.3,81.5,81.4,80.3
Johnson High School,81.3,80.8,80.6,81.2


### Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math (The percentage of students that passed math.)
  * % Passing Reading (The percentage of students that passed reading.)
  * % Overall Passing (The percentage of students that passed math **and** reading.)


In [13]:
#binning
bins = [0, 549.999, 585.999, 615.999, 999999]
group_name = ['< $550', "$550 - 585", "$586 - 615", "> $615"]
school_data_merged['spending_bins'] = pd.cut(school_data_merged['budget']/school_data_merged['size'], bins, labels = group_name)


#group by spending
scores_by_spending = school_data_merged.groupby('spending_bins')

#calculations
avg_math_by_spending = scores_by_spending['math_score'].mean()
avg_reading_by_spending = scores_by_spending['reading_score'].mean()
pass_math_by_spending = school_data_merged[school_data_merged['math_score'] >= 70].groupby('spending_bins')['Student ID'].count()/scores_by_spending['Student ID'].count()
pass_reading_by_spending = school_data_merged[school_data_merged['reading_score'] >= 70].groupby('spending_bins')['Student ID'].count()/scores_by_spending['Student ID'].count()
overall_rate_by_spending = school_data_merged[(school_data_merged['reading_score'] >= 70) & (school_data_merged['math_score'] >= 70)].groupby('spending_bins')['Student ID'].count()/scores_by_spending['Student ID'].count()

            
# df build            
scores_by_spend = pd.DataFrame({
    "Average Math Score": avg_math_by_spending,
    "Average Reading Score": avg_reading_by_spending,
    'Passing Math Rate': pass_math_by_spending,
    'Passing Reading Rate': pass_reading_by_spending,
    "Overall Passing Rate": overall_rate_by_spending
            
})

scores_by_spend.index.name = "Per Student Budget"
scores_by_spend = scores_by_spend.reindex(group_name)

#format
scores_by_spend.style.format({'Average Math Score': '{:.1f}', 
                              'Average Reading Score': '{:.1f}', 
                              'Passing Math Rate': '{:.1%}', 
                              'Passing Reading Rate':'{:.1%}', 
                              'Overall Passing Rate': '{:.1%}'})

,Average Math Score,Average Reading Score,Passing Math Rate,Passing Reading Rate,Overall Passing Rate
Per Student Budget,,,,,
< $550,nan,nan,nan%,nan%,nan%
$550 - 585,83.4,84.0,93.7%,96.7%,90.6%
$586 - 615,83.5,83.8,94.1%,95.9%,90.1%
> $615,77.6,81.3,69.3%,82.6%,57.5%


### Scores by School Size

* Repeat the above breakdown, but this time group schools based on a reasonable approximation of school size (Small, Medium, Large).


In [14]:
bins = [0, 999, 1999, 999999]
group_name = ["Small (<1000)", "Medium (1000-2000)" , "Large (>2000)"]
school_data_merged['size_bins'] = pd.cut(school_data_merged['size'], bins, labels = group_name)

#group by spending
school_size = school_data_merged.groupby('size_bins')

#calculations 
avg_math = school_size['math_score'].mean()
avg_read = school_size['math_score'].mean()
pass_math = school_data_merged[school_data_merged['math_score'] >= 70].groupby('size_bins')['Student ID'].count()/school_size['Student ID'].count()
pass_read = school_data_merged[school_data_merged['reading_score'] >= 70].groupby('size_bins')['Student ID'].count()/school_size['Student ID'].count()
overall = school_data_merged[(school_data_merged['reading_score'] >= 70) & (school_data_merged['math_score'] >= 70)].groupby('size_bins')['Student ID'].count()/school_size['Student ID'].count()

            
# df build            
scores_by_size = pd.DataFrame({
    "Average Math Score": avg_math,
    "Average Reading Score": avg_read,
    'Passing Math Rate': pass_math,
    'Passing Reading Rate': pass_read,
    "Overall Passing Rate": overall
            
})
            
#reorder columns
scores_by_size = scores_by_size[[
    "Average Math Score",
    "Average Reading Score",
    'Passing Math Rate',
    'Passing Reading Rate',
    "Overall Passing Rate"
]]

scores_by_size.index.name = "Total # of Students"
scores_by_size = scores_by_size.reindex(group_name)

#formating
scores_by_size.style.format({'Average Math Score': '{:.1f}', 
                              'Average Reading Score': '{:.1f}', 
                              'Passing Math Rate': '{:.1%}', 
                              'Passing Reading Rate':'{:.1%}', 
                              'Overall Passing Rate': '{:.1%}'})  

,Average Math Score,Average Reading Score,Passing Math Rate,Passing Reading Rate,Overall Passing Rate
Total # of Students,,,,,
Small (<1000),83.8,83.8,94.0%,96.0%,90.1%
Medium (1000-2000),83.4,83.4,93.6%,96.8%,90.6%
Large (>2000),77.5,77.5,68.7%,82.1%,56.6%


### Scores by School Type

* Repeat the above breakdown, but this time group schools based on school type (Charter vs. District).

In [15]:
# grouped by type
by_type = school_data_merged.groupby("type")

#calculations 
avg_score_by_type = by_type['math_score'].mean()
avg_reading_by_type = by_type['math_score'].mean()
passing_math_by_type = school_data_merged[school_data_merged['math_score'] >= 70].groupby('type')['Student ID'].count()/by_type['Student ID'].count()
passing_read_by_type = school_data_merged[school_data_merged['reading_score'] >= 70].groupby('type')['Student ID'].count()/by_type['Student ID'].count()
overall_by_type = school_data_merged[(school_data_merged['reading_score'] >= 70) & (school_data_merged['math_score'] >= 70)].groupby('type')['Student ID'].count()/by_type['Student ID'].count()

# df build            
scores_by_type = pd.DataFrame({
    "Average Math Score": avg_score_by_type,
    "Average Reading Score": avg_reading_by_type,
    'Passing Math Rate': passing_math_by_type,
    'Passing Reading Rate': passing_read_by_type,
    "Overall Passing Rate": overall_by_type})
    
#reorder columns
scores_by_type = scores_by_type[[
    "Average Math Score",
    "Average Reading Score",
    'Passing Math Rate',
    'Passing Reading Rate',
    "Overall Passing Rate"
]]
scores_by_type.index.name = "Type of School"


#formating
scores_by_type.style.format({'Average Math Score': '{:.1f}', 
                              'Average Reading Score': '{:.1f}', 
                              'Passing Math Rate': '{:.1%}', 
                              'Passing Reading Rate':'{:.1%}', 
                              'Overall Passing Rate': '{:.1%}'})

,Average Math Score,Average Reading Score,Passing Math Rate,Passing Reading Rate,Overall Passing Rate
Type of School,,,,,
Charter,83.4,83.4,93.7%,96.6%,90.6%
District,77.0,77.0,66.5%,80.9%,53.7%


#### Analysis
###### Charter schools seem to perform better over both academic topics, and overall passing rate.
###### Math generally seems to be the underperforming subject.
###### Spending doesn't seem to improve individual, and overall passing rates, though I would like to see how that budget per student is crossed by type of school.